# Creating a basic chat experience with kernel arguments

In this example, we show how you can build a simple chat bot by sending and updating the kernel arguments with your requests.

We introduce the Kernel Arguments object which in this demo functions similarly as a key-value store that you can use when running the kernel.

The chat history is local (i.e. in your computer's RAM) and not persisted anywhere beyond the life of this Jupyter session.

In future examples, we will show how to persist the chat history on disk so that you can bring it into your applications.

In this chat scenario, as the user talks back and forth with the bot, the chat context gets populated with the history of the conversation. During each new run of the kernel, the kernel arguments and chat history can provide the AI with its variables' content.


In [ ]:
# Note: if using a virtual environment, do not run this cell
%pip install -U semantic-kernel
from semantic_kernel import __version__

__version__

Initial configuration for the notebook to run properly.

In [1]:
# Make sure paths are correct for the imports

import os
import sys

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)


sys.path.append(grandparent_dir)

### Configuring the Kernel

Let's get started with the necessary configuration to run Semantic Kernel. For Notebooks, we require a `.env` file with the proper settings for the model you use. Create a new file named `.env` and place it in this directory. Copy the contents of the `.env.example` file from this directory and paste it into the `.env` file that you just created.

**NOTE: Please make sure to include `GLOBAL_LLM_SERVICE` set to either OpenAI, AzureOpenAI, or HuggingFace in your .env file. If this setting is not included, the Service will default to AzureOpenAI.**

#### Option 1: using OpenAI

Add your [OpenAI Key](https://openai.com/product/) key to your `.env` file (org Id only if you have multiple orgs):

```
GLOBAL_LLM_SERVICE="OpenAI"
OPENAI_API_KEY="sk-..."
OPENAI_ORG_ID=""
OPENAI_CHAT_MODEL_ID=""
OPENAI_TEXT_MODEL_ID=""
OPENAI_EMBEDDING_MODEL_ID=""
```
The names should match the names used in the `.env` file, as shown above.

#### Option 2: using Azure OpenAI

Add your [Azure Open AI Service key](https://learn.microsoft.com/azure/cognitive-services/openai/quickstart?pivots=programming-language-studio) settings to the `.env` file in the same folder:

```
GLOBAL_LLM_SERVICE="AzureOpenAI"
AZURE_OPENAI_API_KEY="..."
AZURE_OPENAI_ENDPOINT="https://..."
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME="..."
AZURE_OPENAI_TEXT_DEPLOYMENT_NAME="..."
AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME="..."
AZURE_OPENAI_API_VERSION="..."
```
The names should match the names used in the `.env` file, as shown above.

For more advanced configuration, please follow the steps outlined in the [setup guide](./CONFIGURING_THE_KERNEL.md).

We will load our settings and get the LLM service to use for the notebook.

In [2]:
from services import Service

from samples.service_settings import ServiceSettings

service_settings = ServiceSettings()

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = (
    Service.AzureOpenAI
    if service_settings.global_llm_service is None
    else Service(service_settings.global_llm_service.lower())
)
print(f"Using service type: {selectedService}")

Using service type: Service.AzureOpenAI


In [3]:
from semantic_kernel import Kernel

kernel = Kernel()

service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    service_id = "default"
    kernel.add_service(
        OpenAIChatCompletion(
            service_id=service_id,
        ),
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    service_id = "default"
    kernel.add_service(
        AzureChatCompletion(
            service_id=service_id,
        ),
    )

Let's define a prompt outlining a dialogue chat bot.


In [4]:
prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

Register your semantic function


In [5]:
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings, OpenAIChatPromptExecutionSettings
from semantic_kernel.prompt_template import PromptTemplateConfig
from semantic_kernel.prompt_template.input_variable import InputVariable

if selectedService == Service.OpenAI:
    execution_settings = OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-3.5-turbo",
        max_tokens=2000,
        temperature=0.7,
    )
elif selectedService == Service.AzureOpenAI:
    execution_settings = AzureChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-35-turbo",
        max_tokens=2000,
        temperature=0.7,
    )

prompt_template_config = PromptTemplateConfig(
    template=prompt,
    name="chat",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="user_input", description="The user input", is_required=True),
        InputVariable(name="history", description="The conversation history", is_required=True),
    ],
    execution_settings=execution_settings,
)

chat_function = kernel.add_function(
    function_name="chat",
    plugin_name="chatPlugin",
    prompt_template_config=prompt_template_config,
)

In [6]:
from semantic_kernel.contents import ChatHistory

chat_history = ChatHistory()
chat_history.add_system_message("You are a helpful chatbot who is good about giving book recommendations.")

Initialize the Kernel Arguments


In [7]:
from semantic_kernel.functions import KernelArguments

arguments = KernelArguments(user_input="Hi, I'm looking for book suggestions", history=chat_history)

Chat with the Bot


In [8]:
response = await kernel.invoke(chat_function, arguments)
print(response)

Sure! I'd be happy to help. Can you tell me a bit more about what you're looking for? Are you into a specific genre, like mystery, fantasy, romance, or non-fiction? Or do you want something similar to a book or author you already love? Let me know, and I'll tailor my recommendations!


Update the history with the output


In [9]:
chat_history.add_assistant_message(str(response))

Keep Chatting!


In [10]:
async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")

    # Process the user message and get an answer
    answer = await kernel.invoke(chat_function, KernelArguments(user_input=input_text, history=chat_history))

    # Show the response
    print(f"ChatBot: {answer}")

    chat_history.add_user_message(input_text)
    chat_history.add_assistant_message(str(answer))

In [11]:
await chat("I love history and philosophy, I'd like to learn something new about Greece, any suggestion?")

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot: Absolutely! Here are some excellent books that delve into Greek history, philosophy, and culture:

### History:
1. **"The Histories" by Herodotus**  
   Often called the "Father of History," Herodotus offers a fascinating account of ancient Greece, its wars, and its interactions with other civilizations. It's a foundational text for understanding Greek history.

2. **"The Peloponnesian War" by Thucydides**  
   A gripping, firsthand account of the conflict between Athens and Sparta, this book is a must-read for understanding the complexities of Greek politics and warfare.

3. **"A History of Ancient Greece in Fifty Lives" by David Stuttard**  
   This book offers short biographies of influential figures in ancient Greece, giving you a human perspective on its history and achievements.

4. **"Lords of the Sea" by John R. Hale**  
   This book focuses on Athens' rise to power throu

In [12]:
await chat("that sounds interesting, what is it about?")

User: that sounds interesting, what is it about?
ChatBot: Sure! Let me elaborate a bit on some of the books I mentioned, depending on what caught your eye. Here's a quick summary of a few of them:

### History:
1. **"The Histories" by Herodotus**  
   This is essentially the first major work of history ever written! Herodotus explores the events leading up to the Greco-Persian Wars, retelling fascinating stories of battles, cultural customs, and the interplay between East and West. While it's a historical account, it often reads like a collection of adventure tales filled with myths and legends.

2. **"The Peloponnesian War" by Thucydides**  
   This book is more analytical and serious than Herodotus' work. Thucydides, an Athenian historian, meticulously chronicles the decades-long conflict between Athens and Sparta in the 5th century BCE. It's packed with insights on power, politics, and human nature, making it timeless for readers interested in the rise and fall of civilizations.

--

In [13]:
await chat("if I read that book, what exactly will I learn about Greek history?")

User: if I read that book, what exactly will I learn about Greek history?
ChatBot: Great question! If you read one of the history-focused books I mentioned, here's what you'll learn about Greek history, depending on the title you choose:

### 1. **"The Histories" by Herodotus**
   - **What You'll Learn**:  
     - The origins and events of the Greco-Persian Wars, including famous battles like Marathon, Thermopylae, and Salamis.
     - The cultural differences between the Greeks and other ancient civilizations, such as the Persians and Egyptians.
     - Myths, legends, and anecdotes that give insight into Greek beliefs and storytelling traditions.
     - Early ideas about geography, politics, and human behavior, as interpreted by Herodotus.
   - **Why It's Important**:  
     You'll gain a broad understanding of how the Greeks saw themselves in relation to other cultures and how their world was shaped by conflict and cooperation.

---

### 2. **"The Peloponnesian War" by Thucydides**
  

In [14]:
await chat("could you list some more books I could read about this topic?")

User: could you list some more books I could read about this topic?
ChatBot: Of course! Here’s a more extensive list of books that dive into Greek history, philosophy, and culture. These cover a variety of angles—history, politics, mythology, philosophy, and daily life in ancient Greece. Hopefully, you’ll find something new and exciting here:

---

### **Greek History**
1. **"The Greeks: A Global History" by Roderick Beaton**  
   A sweeping, accessible history of the Greeks from ancient times to the modern day, exploring how Greek ideas influenced the world.

2. **"The Rise and Fall of Classical Greece" by Josiah Ober**  
   This book examines how ancient Greece rose to prominence through its unique political and social systems and why it eventually fell.

3. **"Alexander the Great" by Robin Lane Fox**  
   A definitive biography of Alexander the Great, detailing his conquests, ambitions, and the impact of his empire on the ancient world.

4. **"Persian Fire: The First World Empire an

After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!


In [15]:
print(chat_history)

<chat_history><message role="system"><text>You are a helpful chatbot who is good about giving book recommendations.</text></message><message role="assistant"><text>Sure! I'd be happy to help. Can you tell me a bit more about what you're looking for? Are you into a specific genre, like mystery, fantasy, romance, or non-fiction? Or do you want something similar to a book or author you already love? Let me know, and I'll tailor my recommendations!</text></message><message role="user"><text>I love history and philosophy, I'd like to learn something new about Greece, any suggestion?</text></message><message role="assistant"><text>Absolutely! Here are some excellent books that delve into Greek history, philosophy, and culture:

### History:
1. **"The Histories" by Herodotus**  
   Often called the "Father of History," Herodotus offers a fascinating account of ancient Greece, its wars, and its interactions with other civilizations. It's a foundational text for understanding Greek history.

2.